## 1. Import packages

In [1]:
# Import packages
import hail as hl
from bokeh.io import output_notebook,show
import gnomad.utils.vep
from hail.ggplot import *
import plotly
import plotly.io as pio
pio.renderers.default='iframe'

In [48]:
%run -i ~/BroadIS/utils/01_maps_funcs.py

## 2. Import data

In [3]:
# Import gnomaAD v.3.1.2
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')
ht = ht.head(100000) # Subset the data

# Import mutation rates from gnomAD paper
ht_mu = hl.import_table('data/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)
ht_mu = ht_mu.key_by('context', 'ref', 'alt') # has to have a key in order to join using foreign key

# Import context table from gnomad (https://broadinstitute.github.io/gnomad_methods/api_reference/utils/vep.html?highlight=context#gnomad.utils.vep.get_vep_context)
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()
context_table_parsed = context_table.select(context_table.context)
context_table_parsed = context_table_parsed.transmute(context = context_table_parsed.context[2:5])

Initializing Hail with default parameters...


2022-09-27 21:56:23 WARN  Utils:69 - Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface en0)
2022-09-27 21:56:23 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-09-27 21:56:24 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-09-27 21:56:26 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20220927-2156-0.2.99-57537fea08d4.log
2022-09-27 21:56:40 Hail: INFO: Reading table to impute column types
2022-09-27 21:56:42 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


### Show the data structure

In [4]:
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [5]:
# Table with methylation level and mutational rate in the trinucleotide context
ht_mu.show(3)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09


In [6]:
# This table contains already precalculated nucleotides -3/+3 from mutation site 
context_table_parsed.show(3)

,,
locus,alleles,context
locus<GRCh38>,array<str>,str
chr1:10001,"[""T"",""A""]","""NTA"""
chr1:10001,"[""T"",""C""]","""NTA"""
chr1:10001,"[""T"",""G""]","""NTA"""


## 3. Add context field to main data

In [7]:
# Before joining the tri-nucleotide context of mutation
ht.count()

100000

In [8]:
# Join only matching rows from context to ht table.
#ht = ht.key_by('locus', 'alleles').join(context_table_parsed.key_by('locus', 'alleles'), how = 'left') # resorts the data making it slow
ht = ht.annotate(**context_table_parsed[ht.locus, ht.alleles])


In [9]:
# After
ht.count()

100000

## 4. Add mutation rates for added contexts

In [10]:
# Split alleles field to ref and alt allele
ht = ht.annotate(ref=ht.alleles[0], alt=ht.alleles[1])

# Add mutation rates according to the context, but also ref and alt allele for this context
#ht = ht.key_by("context", "ref", "alt").join(ht_mu.key_by("context", "ref", "alt"), how = 'left') # resorts the data making it slow
ht = ht.annotate(**ht_mu[ht.context, ht.ref, ht.alt])


In [11]:
# After adding context and mutation rates to the main table 
# (can be more than original number of rows as context may occure more than once depending on the locus)
ht.count()

100000

In [12]:
# Show that contexts may be the same, but locus is completely different
ht.show(3)

2022-09-27 22:00:40 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:00:41 Hail: INFO: Coerced sorted dataset
2022-09-27 22:02:21 Hail: INFO: Ordering unsorted dataset with network shuffle4]


+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

## 5. Train linear model on synonymous variants for mutational class correction

In [29]:
ht_syn_ps = train_on_synonymous_per_gene(ht)

2022-09-27 22:46:48 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'faf_meta' -> 'faf_meta_1'
    'faf_index_dict' -> 'faf_index_dict_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'filtering_model' -> 'filtering_model_1'
    'vep_version' -> 'vep_version_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'age_distribution' -> 'age_distribution_1'
    'freq_meta' -> 'freq_meta_1'
    'dbsnp_version' -> 'dbsnp_version_1'


In [30]:
ht_syn_ps.show(3)

2022-09-27 22:47:46 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:47:47 Hail: INFO: Coerced sorted dataset
2022-09-27 22:48:31 Hail: INFO: Coerced sorted dataset>           (11 + 3) / 14]
2022-09-27 22:49:29 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:49:30 Hail: INFO: Coerced sorted dataset
2022-09-27 22:50:54 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]


,,,,,,,
context,ref,alt,mu_snp,gene_id,N_variants,N_singletons,ps
str,str,str,float64,array<str>,int64,int64,float64
"""AAT""","""A""","""G""",7.75e-09,"[""ENSG00000186092"",""ENSG00000186092"",""79501""]",1,1,1.00e+00
"""ACA""","""C""","""T""",9.32e-09,"[""ENSG00000186092"",""ENSG00000186092"",""79501""]",1,NA,NA
"""ACA""","""C""","""T""",9.32e-09,"[""ENSG00000230021"",""ENSG00000230021"",""ENSG00000235146"",""ENSG00000235146"",""ENSG00000230021"",""105378947""]",1,NA,NA


### Show input table for regression

In [31]:
# How many rows after adding mutation rates
ht_syn_ps.count()

2022-09-27 22:51:40 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:51:40 Hail: INFO: Coerced sorted dataset
2022-09-27 22:52:25 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-27 22:53:06 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:53:07 Hail: INFO: Coerced sorted dataset
2022-09-27 22:53:46 Hail: INFO: Coerced sorted dataset====>       (12 + 2) / 14]


97

### Perform regression

In [33]:
# Perform regression
ht_syn_lm = ht_syn_ps.aggregate(hl.agg.linreg(ht_syn_ps.ps, [1, ht_syn_ps.mu_snp], weight=ht_syn_ps.N_variants).beta)

# Show intercept and beta
ht_syn_lm

2022-09-27 22:57:05 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:57:05 Hail: INFO: Coerced sorted dataset
2022-09-27 22:57:46 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-27 22:58:29 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-27 22:58:30 Hail: INFO: Coerced sorted dataset
2022-09-27 22:59:12 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]


[0.8785356978109927, -7363214.666563801]

## 6. Predict expected number of variants for each context

### Function for regression eventually will be made starting here and put in `/utils/utils.py` script

In [49]:
maps_table = regress_per_context_per_gene(ht, ht_syn_lm)

In [50]:
maps_table.show(maps_table.count())

2022-09-28 00:08:31 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:08:31 Hail: INFO: Coerced sorted dataset
2022-09-28 00:09:14 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:09:18 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:10:02 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:10:02 Hail: INFO: Coerced sorted dataset
2022-09-28 00:10:49 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:11:36 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:11:36 Hail: INFO: Coerced sorted dataset
2022-09-28 00:12:30 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:12:35 Hail: INFO: Ordering unsorted dataset with network shuffle4]


,,,,,,,
consequence,gene_id,N_singletons,expected_singletons,N_variants,ps_agg,maps,maps_sem
str,array<str>,int64,float64,int64,float64,float64,float64
"""3_prime_UTR_variant""","[""ENSG00000186092"",""ENSG00000186092"",""79501""]",67,1.61e+02,415,1.61e-01,-2.27e-01,1.81e-02
"""3_prime_UTR_variant""","[""ENSG00000230021"",""ENSG00000230021"",""ENSG00000235146"",""ENSG00000235146"",""ENSG00000230021"",""105378947""]",2,1.19e+01,36,5.56e-02,-2.74e-01,3.82e-02
"""3_prime_UTR_variant""","[""ENSG00000236601"",""ENSG00000228463"",""ENSG00000236601"",""ENSG00000236601"",""112268260"",""112268260"",""112268260"",""112268260"",""112268260"",""112268260"",""112268260"",""112268260"",""112268260""]",21,8.44e+01,241,8.71e-02,-2.63e-01,1.82e-02
"""5_prime_UTR_variant""","[""ENSG00000186092"",""ENSG00000240361"",""ENSG00000186092"",""ENSG00000240361"",""79501""]",0,8.19e-01,5,0.00e+00,-1.64e-01,0.00e+00
"""downstream_gene_variant""","[""100996442"",""100996442"",""100996442"",""100996442"",""100996442"",""100996442""]",25,5.13e+01,304,8.22e-02,-8.66e-02,1.58e-02
"""downstream_gene_variant""","[""ENSG00000186092""]",106,3.02e+02,709,1.50e-01,-2.76e-01,1.34e-02
"""downstream_gene_variant""","[""ENSG00000186092"",""ENSG00000186092""]",12,1.95e+01,42,2.86e-01,-1.79e-01,6.97e-02
"""downstream_gene_variant""","[""ENSG00000186092"",""ENSG00000186092"",""79501""]",292,6.59e+02,1764,1.66e-01,-2.08e-01,8.85e-03


In [51]:
ggplot(maps_table, aes(x=maps_table.gene_id, y = maps_table.maps)) + geom_col(aes(fill=maps_table.consequence))


2022-09-28 00:13:26 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:13:26 Hail: INFO: Coerced sorted dataset
2022-09-28 00:14:24 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:15:11 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:15:11 Hail: INFO: Coerced sorted dataset
2022-09-28 00:15:57 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-28 00:16:02 Hail: INFO: Ordering unsorted dataset with network shuffle4]
